# TASK 1

## Task: Update a factory automation component

Welcome to your first task as the factory automation operator!

We currently have a Cisco Kinetic EFM instance running in our factory edge. It's already pulling data from our 3D printer for monitoring and control perposes.

<Image of Cisco EFM Designer>

We have a new version that provides new features and needed functionality for the business, in our case, a brand new monitoring dashboard for the factory floor team.

Lets upgrade our component using the Kubernetes API!

## How to play

Some input below will be needed, fields may be missing, read the comments to work out what needs completing to sucessfully remediate the issue!

To run a block of code below, hold SHIFT+ENTER. 
NO NEW LINES OF CODE ARE NEEDED, WE'RE NOT THAT MEAN!


In [6]:
# See the current deployment in our edge kubernetes cluster using the Kubernets CLI
! kubectl get deployments



No resources found.


## Kubernetes Deployments 

Deployments tell kubernetes to run one or more copies of the same Docker container image, known as replicas, they also provide logic to perform "rolling upgrades" on deployments with many replicas, allowing to move to a new version of a given containerized application without downtime for the users.

We can see the indevidual pods (container instances) powering a given deployment by running `kubectl describe deployment <deployment name>`

The deployment we care about is `EFM-Kinetic-Printer1`....


In [9]:
# Try inspecting the deployment we want to update (you'll need the name...)
! kubectl describe deployment ...


Error from server (NotFound): deployments.extensions "..." not found


## Kubernetes Services

You may have noticed in the output above, each pod had an internal IP within the kubernetes cluster, this is no good for instances of a single app all running concurrently for resilience and handling load (which is what a `Deployment` construct in kubernetes is designed for!)

So we'll need something to put a public/external IP address on the pool of pods and load balance access between them. In kubernetes, these are known as services.

In [28]:
# List the kubernetes service objects in our factory cluster.
! kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


We have a few services already, notice our `EFM-Kinetic-Printer1-Service` 
(these names are arbitrary and upto the person automating the deployments and services against the cluster).

We can see we have an external IP, we could select just that service as below..

In [29]:
# show just one service
! kubectl get service EFM-Kinetic-Printer1-Service

# We can also ask for kubectl output in JSON, allowing us to filter the bits we want easily!
! kubectl get service EFM-Kinetic-Printer1-Service -o json | jq -r '.spec.clusterIP'

# Now we know howto get the IP, we can easily give you the EFM UI URL for your environment...
! echo "https://`kubectl get service EFM-Kinetic-Printer1-Service -o json | jq -r .spec.clusterIP`/dataflow.html"

Error from server (NotFound): services "EFM-Kinetic-Printer1-Service" not found
Error from server (NotFound): services "EFM-Kinetic-Printer1-Service" not found
Error from server (NotFound): services "EFM-Kinetic-Printer1-Service" not found
https:///dataflow.html


## Access the EFM via it's service IP.

Click the URL above to open the EFM UI to see the Cisco Kinetic UI!

Log in with User: `EFMAdmin` and Password: `cisco123`.

![~/CHALLENGES/.images/efm-login.png]()


In [ ]:
%%bash

echo "New cluster info:"
cat ./temp/output.clustercreate | jq -r '.name, .state, .uuid, .ingress_vips[]'

In [ ]:
%%bash

# Get current list of users
echo "Current System Users:"
curl -k -b ./temp/cookie.txt https://$MGMT_HOST/2/localusers | jq -r '.[]|.UserName'

# Create a user template for our factory
cat <<EOF > ./temp/user.json
{
  "UserName": "${NEW_USER}",
  "FirstName": "",
  "LastName": "",
  "Password": "${NEW_USER_PASSWORD}",
  "Disable": false,
  "Role": "User"
}
EOF

# Create local authz template for binding new user to our cluster (no LDAP integration)
cat <<EOF > ./temp/auth_local.json
{
  "name": "${NEW_USER}",
  "local": true
 }

EOF

# Create New User

curl -k -b ./temp/cookie.txt -H "Content-Type:application/json" -X POST -d @./temp/user.json https://$MGMT_HOST/2/localusers

echo "Current System Users after create:"
curl -k -b ./temp/cookie.txt https://$MGMT_HOST/2/localusers | jq -r '.[]|.UserName'


echo "Get the cluster UUID for ${NEW_CLUSTER_NAME} :"
cluster_uuid=`curl -k -b ./temp/cookie.txt https://$MGMT_HOST/2/clusters| jq -r '.[]|.name, .uuid' | grep -A1 ${NEW_CLUSTER_NAME} | sed -n 2p`
echo ${cluster_uuid} 
echo ${cluster_uuid} > .clusteruuid
cat .clusteruuid

echo "Add user binding to cluster using UUID..."
curl -sk -b ./temp/cookie.txt -X POST -H "Content-Type: application/json" -d @./temp/auth_local.json https://$MGMT_HOST/2/clusters/${cluster_uuid}/authz




In [ ]:
%%bash
# Switching API context to new user.
# Pulling KubeConfig and Kubernetes API endpoint details

rm -Rf ./temp ~/.kube
mkdir ./temp ~/.kube/
cluster_uuid=`cat .clusteruuid`

# Auth to API as new user, get auth token as cookie."
curl -k -c ./temp/cookie.txt -H "Content-Type:application/x-www-form-urlencoded" -d "username=${NEW_USER}&password=${NEW_USER_PASSWORD}" https://$MGMT_HOST/2/system/login/

# Download our clusters kubeconfig file
curl -k -X GET -b ./temp/cookie.txt https://${MGMT_HOST}/2/clusters/${cluster_uuid}/env > ~/.kube/config

In [ ]:
# Test Kubeconfig access to Kubernetes Cluster
! kubectl get nodes
